In [4]:
# Step 1: Install required libraries
!pip install transformers datasets sentencepiece scikit-learn


In [5]:
# Step 2: Import libraries
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [19]:
# Step 3: Prepare your dataset
# Example: 50 old sentences + modern equivalents
old_sentences = [
    "Bir çox sifətlər var insanda hələ",
"Almasan yaxşıdır onları dilə",
"Pis sözü ağzına alınca insan",
"Qarnında öldürsə yaxşıdır, inan",
"Bu dünya dolanıb döndükcə ruzgar",
"Qoy səndən yaxşı söz qalsın yadigar",

    "Harda ki eşq əli bir süfrə açar",
"Bu gözəl hekayə ona duz saçar",
"Bu qədər nazəndə, incə bir dilbər",
"Nədən çılpaq qalsın bu vaxta qədər",
"Onu bəzəməmiş arif əlləri",
"Odur ki, çılpaqdır o gözəl pəri",

"Daim xaraba da olsa bir diyar",
"Bil ki, abadlıqdan onda nişan var",
"İstər xarab olsun, istərsə abad",
"Daimi bünövrə qoymamış həyat",

"Ey varlıq nəqşinin naziri olan",
"Qaldır əngəlləri idrak yolundan",
"Boş təbli döyərək hay-haray salma",
"Heç kəsin səsinə biganə qalma",
"Bir toz görünsə də hər zərrə əgər",
"Dünya pərdəsində o da iş görər",

    "Qısa söz; dünyada nə sən, nə də mən",
"Oyuncaq deyilik xilqətimizdən",
"Nə şəhvət, nə yuxu, nə də ki yemek",
"Həyatın mənası olmasın gərək",
"Yatmağı, yeməyi bu aləmdə sən",
"Eşşək, öküzdə də görə bilərsən",
"Təbiət quranda xilqətimizi",
"Başqa səhifədə yazmışdır bizi",

    "İlahi, dar gündə sən ol həmdəmim",
    "İstədiyin yerə çatsın qədəmim",
"Söylə nə vaxtadək bir tikə üçün" ,
    "Şaha və gədaya yalvarım hər gün",

    "Məcnun bəslədikcə o heyvanları",
"Çəkdi dövrəsinə bir can hasarı",
"Hasar ətrafında gəzən hər heyvan",
"Məcnunun işinə yaradı hər an",
"Məcnuna bənzəsən insanlıqda sən",
"Dünyada nə qüssə, nə qəm çəkərsən",

    "Ata öz oğlunu əzizləyən tək",
"Məcnun da maralı əzizləyərək",
"Tutub sığalladı o dildarını",
"Məni pis taleyim təqib edir, bil",
"Heç kəs bədbəxtlikdən qurtaran deyil",
"Sən də mi yarından düşdün aralı",
"Ey çöl qoşununun gözəl sarbanı",
"Dağlar döşündəki çadırın hanı",
"Xoş ətrin canandan gətirir xəbər",
"Gözün sevgilimin gözünə bənzər",


]
modern_sentences = [
    "İnsanda hələ çox sifətlər var",
    "Onları dilə gətirməsən yaxşıdır",
    "İnsan pis söz demək istəyəndə",
    "Qarnında boğub saxlaması daha yaxşıdır",
    "Dünya fırlanıb dolandıqca ruzgar",
    "Qoy səndən xoş söz yadigar qalsın",

    "Harada ki eşq süfrə açır",
    "Gözəl bir hekayə ona dad qatır",
    "Bu qədər nazlı, incə bir gözəl",
    "Niyə indiyə qədər bəzəksiz qalsın",
    "Onu arif əllər bəzəməyib",
    "Ona görə də çılpaqdır o gözəl pəri",

    "Bir diyar daim xaraba görünsə də",
    "Bil ki, onda abadlığın izi var",
    "İstər xaraba olsun, istər abad",
    "Həyat heç vaxt daimi bünövrə qoymayıb",

    "Ey varlığın gözəlliyini qoruyan",
    "Qaldır maneələri idrak yolundan",
    "Boş təbil çalıb hay-küy salma",
    "Heç kimin səsinə laqeyd qalma",
    "Bir zərrə toz görünsə belə",
    "Dünya səhnəsində onun da rolu var",

    "Qısaca desək: nə sən, nə də mən",
    "Xilqətimizdən oyuncaqlar deyilik",
    "Nə şəhvət, nə yuxu, nə də yemək",
    "Həyatın mənası olmamalıdır",
    "Yatmaqla, yeməklə bu dünyada",
    "Eşşəkdə, öküzdə də rastlaşa bilərsən",
    "Təbiət yaradanda xilqətimizi",
    "Bizim üçün ayrıca səhifə yazıb",

    "İlahi, çətin gündə sən mənim həmdəmim ol",
    "Addımlarım istədiyin yerə çatsın",
    "De görüm, nə vaxta qədər bir tikə çörək üçün",
    "Hər gün həm şahdan, həm də kasıbdan dilənim",

    "Məcnun heyvanları bəslədikcə",
    "Onların ətrafında bir canlı hasar qurdu",
    "Hasarın ətrafında gəzən hər heyvan",
    "Məcnunun işinə hər an yaradı",
    "Əgər sən də insanlıqda Məcnuna bənzəsən",
    "Dünyada nə qüssə, nə də kədər çəkərsən",

    "Ata öz oğlunu necə əzizləyirsə",
    "Məcnun da maralı sevərək sığallayırdı",
    "O, sevimlisini tutub oxşayırdı",
    "Amma məni pis tale izləyir, bil",
    "Heç kəs bədbəxtlikdən qurtula bilmir",
    "Sən dəmi sevgilindən ayrı düşmüsən",
    "Ey səhrada karvanı aparan gözəl sarban",
    "Dağların döşündəki çadırın hanı",
    "Xoş ətrin sevgilidən xəbər gətirir",
    "Gözlərin sevgilimin gözlərinə bənzəyir"
]



In [20]:
# Step 4: Load sentiment analysis models
xlmr_model = pipeline("sentiment-analysis", model="xlm-roberta-base")
bert_model = pipeline("sentiment-analysis", model="bert-base-multilingual-cased")


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [21]:
# Step 5: Run sentiment analysis
xlmr_results = [xlmr_model(s)[0] for s in modern_sentences]
bert_results = [bert_model(s)[0] for s in modern_sentences]

print("XLM-R Sentiment Results:", xlmr_results[:5])
print("BERT Sentiment Results:", bert_results[:5])


XLM-R Sentiment Results: [{'label': 'LABEL_1', 'score': 0.5271710157394409}, {'label': 'LABEL_1', 'score': 0.5242962837219238}, {'label': 'LABEL_1', 'score': 0.5247834324836731}, {'label': 'LABEL_1', 'score': 0.5274254679679871}, {'label': 'LABEL_1', 'score': 0.5271636843681335}]
BERT Sentiment Results: [{'label': 'LABEL_0', 'score': 0.5282865166664124}, {'label': 'LABEL_0', 'score': 0.5259682536125183}, {'label': 'LABEL_0', 'score': 0.5024901032447815}, {'label': 'LABEL_0', 'score': 0.532522439956665}, {'label': 'LABEL_0', 'score': 0.5314029455184937}]


In [22]:
# Step 6: Convert labels to numeric for metrics
def label_to_num(label):
    return 1 if label.lower() in ["positive"] else 0

y_true = [
 1,1,0,0,1,1,
 1,1,1,0,0,0,
 0,1,0,0,
 1,1,0,1,1,1,
 0,0,0,0,0,0,1,1,
 0,1,0,0,
 1,1,1,1,1,1,
 1,1,1,0,0,0,1,0,1,1
]


y_pred_xlmr = [label_to_num(r['label']) for r in xlmr_results]
y_pred_bert = [label_to_num(r['label']) for r in bert_results]

In [23]:
# Step 7: Calculate metrics
print("XLM-R Metrics:")
print("Precision:", precision_score(y_true, y_pred_xlmr))
print("Recall:", recall_score(y_true, y_pred_xlmr))
print("F1:", f1_score(y_true, y_pred_xlmr))

print("BERT Metrics:")
print("Precision:", precision_score(y_true, y_pred_bert))
print("Recall:", recall_score(y_true, y_pred_bert))
print("F1:", f1_score(y_true, y_pred_bert))




XLM-R Metrics:
Precision: 0.0
Recall: 0.0
F1: 0.0
BERT Metrics:
Precision: 0.0
Recall: 0.0
F1: 0.0


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:

# Step 8: Sentence similarity
# Use embeddings from XLM-R
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base")

def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.logits.detach().numpy()

embeddings = [get_embedding(s) for s in modern_sentences]
similarity_matrix = cosine_similarity(np.vstack(embeddings))

print("Similarity Matrix Shape:", similarity_matrix.shape)
print("Similarity between sentence 1 and 2:", similarity_matrix[0][1])


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Similarity Matrix Shape: (50, 50)
Similarity between sentence 1 and 2: 1.0


In [26]:
# Step 8: Sentence similarity between old and modern sentences
from transformers import AutoModel

import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Use XLM-R base model for embeddings
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModel.from_pretrained("xlm-roberta-base")

def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Mean pooling of last hidden state
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Get embeddings for old and modern sentences
old_embeddings = [get_embedding(s) for s in old_sentences]
modern_embeddings = [get_embedding(s) for s in modern_sentences]

# Compare each old sentence with its modern equivalent
for i in range(len(old_sentences)):
    sim = cosine_similarity(old_embeddings[i], modern_embeddings[i])[0][0]
    print(f"Similarity between old and modern sentence {i+1}: {sim:.4f}")



Similarity between old and modern sentence 1: 0.9995
Similarity between old and modern sentence 2: 0.9975
Similarity between old and modern sentence 3: 0.9962
Similarity between old and modern sentence 4: 0.9984
Similarity between old and modern sentence 5: 0.9957
Similarity between old and modern sentence 6: 0.9997
Similarity between old and modern sentence 7: 0.9972
Similarity between old and modern sentence 8: 0.9976
Similarity between old and modern sentence 9: 0.9991
Similarity between old and modern sentence 10: 0.9977
Similarity between old and modern sentence 11: 0.9989
Similarity between old and modern sentence 12: 0.9990
Similarity between old and modern sentence 13: 0.9963
Similarity between old and modern sentence 14: 0.9983
Similarity between old and modern sentence 15: 0.9991
Similarity between old and modern sentence 16: 0.9989
Similarity between old and modern sentence 17: 0.9969
Similarity between old and modern sentence 18: 0.9987
Similarity between old and modern sen